## Table of Contents
* [Descripción del Caso](#scrollTo=nae5HeOE4LRa)
    * [Apertura del archivo con los datos](#scrollTo=Si0hHUA5Bfdx)
        
* [1. Exploración de datos](#scrollTo=Fy9DrWiM4LRj)
* [2. Transformación de datos](#scrollTo=aPmg8oDG4LRs)
* [3. Trabajar con valores ausentes](#scrollTo=0MC3qt6L4LRy)
    * [3.1. Restaurar valores ausentes en `total_income`](#scrollTo=lOnPEOKt4LRy)
    * [3.2. Restaurar valores en `days_employed`](#scrollTo=fMiTmANz4LR0)
* [4. Clasificación de datos](#scrollTo=PZHTsD254LR2)
* [5. Comprobación de las hipótesis](#scrollTo=f-PwCzud4LR4)
* [Conclusión general](#scrollTo=HkGsIZYu4LR6)

















# Descripción del Caso
##Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

**Preguntas a responder:**



*   ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
*   ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
*   ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
*   ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?







[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)


## Apertura del archivo con los datos



In [ ]:
# Cargar todas las librerías
import pandas as pd
#Cargando drive, ya que el proyecto fue realizado en Google Colab y el archivo se encuentra en Google Drive
from google.colab import drive 
drive.mount('/content/gdrive')

# Carga los datos
try:
  #leyendo mi archivo que se encuentra en Google Drive
  cc_scoring = pd.read_csv("gdrive/My Drive/Education/Practicum/Sprint 2/Project 2/credit_scoring_eng.csv", sep=",")
except:
  #leyendo el archivo para el revisor de código de Practicum
  cc_scoring = pd.read_csv("/datasets/credit_scoring_eng.csv", sep=",")

Mounted at /content/gdrive


# 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)

In [ ]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
cc_scoring.shape


(21525, 12)

In [ ]:
# vamos a mostrar las primeras 10 filas 
cc_scoring.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


A primera vista se puede observar lo siguiente:
1. En la columna **days_employed** existen valores negativos, y valores excesivamente grandes. Los valores grandes probablemente sean horas y no días.
2. En la columna **education** existen datos escritos iguales pero escritos en mayúsculas, minúsculas, o una combinación de ambas. Se debe escribir todo con minúsculas para facilitar el manejo de los datos.
3. En la columna **purpose** existen propósitos similares, pero escritos de formas diferentes

In [ ]:
# Obtener información sobre los datos
cc_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
#valores faltantes en las columnas
cc_scoring.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 Existen valores ausentes en las columnas **days_employed** y **total_income**

In [ ]:
# duplicates
cc_scoring.duplicated().sum()

54

Existen 54 datos duplicados

In [ ]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
cc_scoring_na = cc_scoring.loc[cc_scoring["days_employed"].isna()]
cc_scoring_na.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


In [ ]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
values_na = len(cc_scoring.loc[(cc_scoring["days_employed"].isna())&(cc_scoring["total_income"].isna())])
values_na

2174

In [ ]:
##Procedo a calcular el porcentaje de valores ausentes en comparacion con el conjunto de datos completo
(values_na/len(cc_scoring))*100

10.099883855981417

**Conclusión intermedia**

El número de filas de la tabla filtrada coincide con el número de valores ausentes. Además, los valores ausentes son simétricos en las dos columnnas con vales ausentes. Es decir, las filas con valores ausentes en la columna **days_employed** tambien poseen valores ausentes en la columna **total_income**.

El porcentaje de los valores ausentes en comparación con el conjunto de datos completo es del 10%, por ende, es un valor considerablemente grande.

In [ ]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada "income_type" y la columna con los valores ausentes
print(cc_scoring_na['income_type'].value_counts(normalize=True))

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64


In [ ]:
# Comprobación de la distribución
print(cc_scoring['income_type'].value_counts(normalize=True))

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64


**Posibles razones por las que hay valores ausentes en los datos**

Al comprarar la tabla con valores ausentes y la tabla original sobre la característica identificada "income_type", podemos observar que no existen valores asociados a **unemployed**, **student**, ni a **paternity / maternity leave**


In [ ]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada 'education_id' y la columna con los valores ausentes
print(cc_scoring_na['education_id'].value_counts(normalize=True))



1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64


In [ ]:
# Comprobando la distribución en el conjunto de datos entero
print(cc_scoring['education_id'].value_counts(normalize=True))


1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64


**Conclusión intermedia**

La distribución en el conjunto de datos original es similar a la distribución de la tabla filtrada, sobre la característica identificada 'education_id'. Sin embargo podemos ver que en la tabla filtrada desaparece el "education_id" 4. Lo que significa que no existen valores ausentes en esta categoría.


In [ ]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada 'family_status' y la columna con los valores ausentes
print(cc_scoring_na['family_status'].value_counts(normalize=True))


married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64


In [ ]:
# Comprobando la distribución en el conjunto de datos entero
print(cc_scoring['family_status'].value_counts(normalize=True))

married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64


**Conclusión intermedia**

La distribución en el conjunto de datos original es similar a la distribución de la tabla filtrada, sobre la característica identificada 'family_status'

In [ ]:
cc_scoring.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


**Conclusiones**

1. Hay 21525 registros y 12 atributos
2. `days_employed` y `total_income` tienen 2174 registros faltantes respectivamente.
4. `days_employed` y `total_income` se pueden redondear a los números enteros más cercanos.
5. El mínimo dob_years es 0, lo que debe investigarse. Sin embargo, este campo se puede categorizar.
6. `days_employed` tiene valores negativos. ¿Es esto por diseño?
7. el campo `education` tiene un etiquetado inconsistente.
8. `purpose` parece ser texto libre, que podría tratarse para que sea útil
9. En las bases de datos (databases), para evitar la redundancia, `education` se almacenaría por separado y se vincularía a la tabla **credit report** mediante `education_id`; lo mismo ocurre con `family_status`.
10. Existen 54 registros duplicados.
11. El campo de `children` muestra valores desde -1 hasta 20. Los valores negativos y excesivamente elevados se pueden considerar error de ingreso de datos.
12) En `days_employed` existen valores demasiado elevados, robablemente se trate de horas y no de días.

[1. Exploración de datos](#scrollTo=Fy9DrWiM4LRj)


# 2. Transformación de datos


[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)

In [ ]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
cc_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [ ]:
# Arregla los registros si es necesario
cc_scoring['education'] = cc_scoring['education'].str.lower()

In [ ]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
cc_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Comprobando los valores en la columna `children`

In [ ]:
# Veamos la distribución de los valores en la columna `children`
cc_scoring ['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [ ]:
#Revisemos los datos con 'children'=20
cc_scoring[cc_scoring['children']==20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car
3302,20,NaN,35,secondary education,1,unmarried,4,F,civil servant,0,NaN,profile education
3396,20,NaN,56,bachelor's degree,0,married,0,F,business,0,NaN,university education
3671,20,-913.161503,23,secondary education,1,unmarried,4,F,employee,0,16200.879,buying a second-hand car
3697,20,-2907.910616,40,secondary education,1,civil partnership,1,M,employee,0,18460.911,buying a second-hand car
3735,20,-805.044438,26,bachelor's degree,0,unmarried,4,M,employee,0,21952.103,housing renovation


In [ ]:
##porcentaje de valores problemáticos en la columna "children", children = 20
pchildren20 = len(cc_scoring[cc_scoring ['children']==20])/len(cc_scoring ['children'])
pchildren20

0.0035307781649245064

In [ ]:
##porcentaje de valores problemáticos en la columna "children", children = -1
len(cc_scoring[cc_scoring ['children']==(-1)])/len(cc_scoring ['children'])


0.002183507549361208

Podemos observar que en la columna **"children"** existen valores excesivamente grandes como 20 (0.0035). Y valores negativos (0.00218). Por lo tanto, se procederá a asumir que hubo un error de ingreso de datos, y se realizará las siguientes correcciones:

1.   Se cambiará 20, por 2. Ya que como vemos en la tabla, es imposible que personas con 21, 23 o 26 años tengan 20 hijos.
2.   Y se cambiará -1 por 1.

Nota: Se procedrá a crear una nueva columa `children_fixed` con las correcciones pertinentes, con la finalidad de mantener disponibles los valores iniciales de la columna `children` 




In [ ]:
#Se procede a crear la columna "children_fixed" con las correciones pertinetes. 

cc_scoring["children_fixed"] = cc_scoring["children"]
cc_scoring["children_fixed"].replace(20,2,inplace=True)
cc_scoring["children_fixed"] = cc_scoring["children_fixed"].apply(lambda x: x * -1 if x < 0 else x)

In [ ]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
cc_scoring ["children_fixed"].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children_fixed, dtype: int64

Comprobando los datos en la columna `days_employed`

In [ ]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
cc_scoring['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Se puede observar gran cantidad de valores negativos. Éstos, se deben cambiar a positivos. También se observan valores excesivamente altos, probablemente son horas y no días.

Nota: Se procedrá a crear una nueva columa `days_employed_fixed` con las correcciones pertinentes, con la finalidad de mantener disponibles los valores iniciales de la columna `days_employed` 

In [ ]:
#Pasar los valores negativos de la columna "days_employed" a positivos
cc_scoring["days_employed_fixed"] = cc_scoring["days_employed"]
cc_scoring.loc[cc_scoring["days_employed_fixed"] < 0, "days_employed_fixed"] = -1 * cc_scoring.loc[cc_scoring["days_employed_fixed"] < 0, "days_employed_fixed"]

In [ ]:
##Comprobamos que ya no existan valores negativos en la nueva columna 'days_employed_fixed'
cc_scoring['days_employed_fixed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed_fixed, dtype: float64

In [ ]:
##Porcentaje de valores excesivamente elevamos en la columna 'days_employed_fixed'
len(cc_scoring[cc_scoring ['days_employed_fixed']>16060])/len(cc_scoring ['days_employed_fixed'])


0.1602787456445993

Podemos observar que exiten valores excesivamente altos en la columna `days_employed_fixed`. Asumiremos que una persona trabaja hasta los 60 años, y que pudo haber comenzado a muy temprana edad (16 años). Entonces como máximo un cliente pudo trabajar 44 años (16060 días). Todos los valores por encima del número de días antes indicado, procederán a dividirse entre 24, asumiendo que hubo un error al momento de ingresar los datos; y que se ingresaron horas de trabajo y no días de trabajo.

In [ ]:
# Aborda los valores problemáticos, si existen.
cc_scoring.loc[cc_scoring["days_employed_fixed"] > 16060, "days_employed_fixed"] = cc_scoring.loc[cc_scoring["days_employed_fixed"] > 16060, "days_employed_fixed"] / 24


In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado
cc_scoring['days_employed_fixed'].describe()

count    19351.000000
mean      4637.432534
std       5352.634480
min         24.141633
25%        926.400370
50%       2192.333630
75%       5532.532133
max      16739.808353
Name: days_employed_fixed, dtype: float64

In [ ]:
# Revisamos `dob_years` en busca de valores sospechosos y cuenta el porcentaje
cc_scoring['dob_years'].unique()


array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

En la columna  `dob_years` existen datos con valor igual a 0, lo cual no está bien. Es razonable suponer que los préstamos se desembolsan a personas mayores de cierta edad.


In [ ]:
# Supongamos que la edad mínima es de 18 años. Calculemos cuando datos son menores a 18 años
cc_scoring[cc_scoring['dob_years'] < 18].shape[0]

101

Revisemos la columna `family_status`. en busca de inconvenientes en sus datos

In [ ]:
# Veamos los valores de la columna
cc_scoring['family_status'].unique()


array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

No existen valores problemáticos en la columna `family_status`

Ahora revisemos la columna `gender`

In [ ]:
# Veamos los valores en la columna
cc_scoring['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Vamos a revisar la columna `income_type`

In [ ]:
# Veamos los valores en la columna
cc_scoring['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Veamos is existen duplicados

In [ ]:
# Comprobar los duplicados
cc_scoring.duplicated().sum()


71

Existen 54 datos duplicados. Sin embargo no los voy a borrar, ya que no existe un "identificador único" (unique_id), que me permita verificar que efectivamente los datos son duplicados.

[2. Transformación de datos](#scrollTo=aPmg8oDG4LRs)


# 3. Trabajar con valores ausentes

[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)

### 3.1. Restaurar valores ausentes en `total_income`



Existen 2 columnas con valores ausentes. Para proceder a llenar los valores ausentes en la columna "total_income", crearé una nueva columna con categoría de edad. Luego procederé a llenar los datos incompletos con los valores media o mediana de 'total_income' (según sea lo más apropiado) calculados en cada categoría de edad




In [ ]:
# Vamos a escribir una función que calcule la categoría de edad
def age_groups(age):
    if age < 18:
        return '<18 years'
    if age >=18 and age < 30:
        return '18-29 years'
    if age >=30 and age < 40:
        return '30-39 years'
    if age >=40 and age < 50:
        return '40-49 years'
    if age >=50 and age < 60:
        return '50-59 years'
    return '60+ years'
    

In [ ]:
# Pruebamos si la función funciona bien
age_groups(50)

'50-59 years'

In [ ]:
# Crear una nueva columna basada en la función
cc_scoring['age_groups'] = cc_scoring['dob_years'].apply(age_groups)


In [ ]:
# Comprobamos los valores en la nueva columna
cc_scoring.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_fixed,days_employed_fixed,age_groups
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,1,8437.673028,40-49 years
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1,4024.803754,30-39 years
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,0,5623.422610,30-39 years
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,3,4124.747207,30-39 years
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,0,14177.753002,50-59 years


[Crea una tabla que solo tenga datos sin valores ausentes. Estos datos se utilizarán para restaurar los valores ausentes.]

In [ ]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
cc_scoring.dropna().head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_fixed,days_employed_fixed,age_groups
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,1,8437.673028,40-49 years
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,1,4024.803754,30-39 years
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,0,5623.422610,30-39 years
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,3,4124.747207,30-39 years
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,0,14177.753002,50-59 years


In [ ]:
# Examinamos los valores medios de 'total_income' agrupados por 'age_groups'
cc_scoring.groupby('age_groups')['total_income'].mean()

age_groups
18-29 years    25533.960641
30-39 years    28312.479963
40-49 years    28551.375635
50-59 years    25811.700327
60+ years      23021.639994
<18 years      25334.072890
Name: total_income, dtype: float64

In [ ]:
# Examinamos los valores medianos de 'total_income' agrupados por 'age_groups'
cc_scoring.groupby('age_groups')['total_income'].median()

age_groups
18-29 years    22742.6535
30-39 years    24667.5280
40-49 years    24764.2290
50-59 years    22203.0745
60+ years      19761.4250
<18 years      24387.0700
Name: total_income, dtype: float64

In [ ]:
## Examinamos los valores medianos de 'days_employed' agrupados por 'age_groups'
cc_scoring.groupby('age_groups')['days_employed_fixed'].mean()

age_groups
18-29 years     1243.470812
30-39 years     2103.700591
40-49 years     3065.967424
50-59 years     7537.862837
60+ years      12714.535762
<18 years       4611.266086
Name: days_employed_fixed, dtype: float64

In [ ]:
# Averiguamos el número de valores nulos en la columna "days_employed_fixed" agrupados por "age_groups"
cc_scoring[cc_scoring['days_employed'].isnull()]['age_groups'].value_counts()

30-39 years    565
40-49 years    537
50-59 years    501
18-29 years    299
60+ years      262
<18 years       10
Name: age_groups, dtype: int64

In [ ]:
# Creamos un diccicionario que mapee "age_group" pra el promedio de "total_income"
age_income = cc_scoring.groupby('age_groups')['total_income'].mean().to_dict()
age_income

{'18-29 years': 25533.960641470178,
 '30-39 years': 28312.479963397927,
 '40-49 years': 28551.375635084816,
 '50-59 years': 25811.70032742939,
 '60+ years': 23021.639994235033,
 '<18 years': 25334.07289010989}

In [ ]:
#  Escribimos una función que usaremos para completar los valores ausentes
def replace_missing_values(age,f):
    if age == '<18 years':
        return f[age]
    if age == '18-29 years':
        return f[age]
    if age == '30-39 years':
        return f[age]
    if age == '40-49 years':
        return f[age]
    if age == '50-59 years':
        return f[age]
    return f[age]
        

In [ ]:
# Compruebamos si funciona
#replace_missing_values(22,)

In [ ]:
# Reemplazamos el ingreso faltante con la media en cada categoría de edad
cc_scoring.loc[cc_scoring['total_income'].isnull(), 'total_income'] = (
    cc_scoring.loc[cc_scoring['total_income'].isnull(), 'age_groups'].apply(replace_missing_values, args=(age_income,))
)

In [ ]:
# Comprobamos que ya no existan valores ausentes
cc_scoring['total_income'].isnull().sum()

0

Ahora que he terminado con `total_income`, comprobaré que el número total de valores en esta columna coincida con el número de valores en otras columnas.

In [ ]:
# Comprobar el número de entradas en las columnas
cc_scoring.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21525 non-null  int64  
 1   days_employed        19351 non-null  float64
 2   dob_years            21525 non-null  int64  
 3   education            21525 non-null  object 
 4   education_id         21525 non-null  int64  
 5   family_status        21525 non-null  object 
 6   family_status_id     21525 non-null  int64  
 7   gender               21525 non-null  object 
 8   income_type          21525 non-null  object 
 9   debt                 21525 non-null  int64  
 10  total_income         21525 non-null  float64
 11  purpose              21525 non-null  object 
 12  children_fixed       21525 non-null  int64  
 13  days_employed_fixed  19351 non-null  float64
 14  age_groups           21525 non-null  object 
dtypes: float64(3), int64(6), object(6)
m

###  3.2. Restaurar valores en `days_employed`

[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)

[Piensa en los parámetros que pueden ayudarte a restaurar los valores ausentes en esta columna. Eventualmente, tendrás que averiguar si debes usar valores medios o medianos para reemplazar los valores ausentes. Probablemente llevarás a cabo una investigación similar a la que realizaste cuando restauraste los datos en la columna anterior.]

In [ ]:
# Distribución de las medianas de `days_employed_fixed` en función de los parámetros identificados
cc_scoring.groupby('age_groups')['days_employed_fixed'].median()



age_groups
18-29 years      999.028882
30-39 years     1601.784231
40-49 years     2111.489906
50-59 years     4790.791920
60+ years      14809.848098
<18 years       1759.038033
Name: days_employed_fixed, dtype: float64

In [ ]:
2103/365


5.761643835616439

In [ ]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
cc_scoring.groupby('age_groups')['days_employed_fixed'].mean()

age_groups
18-29 years     1243.470812
30-39 years     2103.700591
40-49 years     3065.967424
50-59 years     7537.862837
60+ years      12714.535762
<18 years       4611.266086
Name: days_employed_fixed, dtype: float64

Voy a utilizar los valores medios, ya que son los valores que más sentido hacen para mi

In [ ]:
# Creamos un diccionario para mapear los valores promedio de `days_employed`
age_days_employed_fixed = cc_scoring.groupby('age_groups')['days_employed_fixed'].mean().to_dict()

In [ ]:
# Reemplazamos el ingreso faltante con la media en cada categoría de edad
cc_scoring.loc[cc_scoring['days_employed_fixed'].isnull(), 'days_employed_fixed'] = (
    cc_scoring.loc[cc_scoring['days_employed_fixed'].isnull(), 'age_groups'].apply(replace_missing_values, args=(age_days_employed_fixed,))
)

In [ ]:
# Comprobamos que ya no existan valores ausentes
cc_scoring['days_employed_fixed'].isnull().sum()

0

Ahora que he terminado con `days_employed_fixed`, comprobaré que el número total de valores en esta columna coincida con el número de valores en otras columnas.

In [ ]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
cc_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21525 non-null  int64  
 1   days_employed        19351 non-null  float64
 2   dob_years            21525 non-null  int64  
 3   education            21525 non-null  object 
 4   education_id         21525 non-null  int64  
 5   family_status        21525 non-null  object 
 6   family_status_id     21525 non-null  int64  
 7   gender               21525 non-null  object 
 8   income_type          21525 non-null  object 
 9   debt                 21525 non-null  int64  
 10  total_income         21525 non-null  float64
 11  purpose              21525 non-null  object 
 12  children_fixed       21525 non-null  int64  
 13  days_employed_fixed  21525 non-null  float64
 14  age_groups           21525 non-null  object 
dtypes: float64(3), int64(6), object(6)
m

# 4. Clasificación de datos



[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)

In [ ]:
# distribución de "education"
cc_scoring['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [ ]:
# valores únicos de "education"
cc_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [ ]:
# Graduate Degree, Primary Education, Some College, Bachelor's Degree, Seconday Education
cc_scoring['education'] = cc_scoring['education'].str.capitalize()

In [ ]:
# proporción de "education" después de limpiar los datos
cc_scoring['education'].value_counts(normalize=True)

Secondary education    0.707689
Bachelor's degree      0.244367
Some college           0.034564
Primary education      0.013101
Graduate degree        0.000279
Name: education, dtype: float64

In [ ]:
# proporción de "family_status"
cc_scoring['family_status'].value_counts(normalize=True)

married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64

In [ ]:
# proporción de "gender"
cc_scoring['gender'].value_counts(normalize=True)

F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64

In [ ]:
# proporción de "income_type"
cc_scoring['income_type'].value_counts(normalize=True)

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

In [ ]:
# distribución de "purpose"
cc_scoring['purpose'].value_counts()

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

In [ ]:
#valores únicos de "purpose"
cc_scoring['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Existen varios valores únicos que se refieren al mismo porpósito. Procederemos a estandarizar cada las etiquetas que se refieren a un mismo propósito.

In [ ]:
def clean_purpose_field(purpose):
    
    keywords = {
                'car': 'Vehicle loan',
                'cars': 'Vehicle loan',
                'education': 'Education loan',
                'educated': 'Education loan',
                'university': 'Education loan',
                'house': 'Housing loan',
                'wedding': 'Personal loan',
                'estate': 'Housing loan',
                'property': 'Housing loan',
                'housing': 'Housing loan'
    }
    match = set()
    for word in purpose.lower().split():
        if word in keywords:
            match.add(keywords[word])
    return list(match)[0] if len(match) > 0 else 'Unknown'

In [ ]:
#key words
# house, car, education, wedding, real estate, property, house renovation
cc_scoring['loan_category'] = cc_scoring['purpose'].apply(clean_purpose_field)

In [ ]:
# checkear el número de valores en la columna "loan_category"
cc_scoring['loan_category'].value_counts().sum()

21525

In [ ]:
# proporción de "loan_category" después de la limpieza
cc_scoring['loan_category'].value_counts(normalize=True)

Housing loan      0.503600
Vehicle loan      0.200465
Education loan    0.186852
Personal loan     0.109082
Name: loan_category, dtype: float64

Ahora tenemos tan sólo 4 categorías en la columna "purpose", que agrupan todos los propósitos

# 5. Comprobación de las hipótesis

[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)

In [ ]:
#Observamos las columnas de mi tabla "cc_scoring" luego del tratamiento de datos realizados en las secciones anteiores 
cc_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_fixed,days_employed_fixed,age_groups,loan_category
0,1,-8437.673028,42,Bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,1,8437.673028,40-49 years,Housing loan
1,1,-4024.803754,36,Secondary education,1,married,0,F,employee,0,17932.802,car purchase,1,4024.803754,30-39 years,Vehicle loan
2,0,-5623.422610,33,Secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,0,5623.422610,30-39 years,Housing loan
3,3,-4124.747207,32,Secondary education,1,married,0,M,employee,0,42820.568,supplementary education,3,4124.747207,30-39 years,Education loan
4,0,340266.072047,53,Secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,0,14177.753002,50-59 years,Personal loan


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [ ]:
# los resultados son idénticos a los que se obtienen con la tabla de referencias cruzadas. necesitaría 
# realizar otro paso para entrar en un formato de tabla
cc_scoring.groupby(['children_fixed', 'debt'])['debt'].count()

children_fixed  debt
0               0       13086
                1        1063
1               0        4420
                1         445
2               0        1929
                1         202
3               0         303
                1          27
4               0          37
                1           4
5               0           9
Name: debt, dtype: int64

In [ ]:
#  el único campo cercano al pago de un préstamo es el campo de la deuda ("debt"); no hay ningún elemento de duración del préstamo
res = pd.crosstab(index=cc_scoring['children_fixed'], 
            columns=cc_scoring['debt']).reset_index().rename(columns={0: 'loan_paid', 
                                                                              1:'loan_default'}
                                                            )
res['percent_paid'] = 100 * res['loan_paid'] / (res['loan_paid'] + res['loan_default'])

In [ ]:
res

debt,children_fixed,loan_paid,loan_default,percent_paid
0,0,13086,1063,92.487102
1,1,4420,445,90.853032
2,2,1929,202,90.520882
3,3,303,27,91.818182
4,4,37,4,90.243902
5,5,9,0,100.000000


In [ ]:
# Calculamos el valor medio en la columna "total_income"
cc_scoring['total_income'].median()

24596.98

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [ ]:
res = pd.crosstab(index=cc_scoring['family_status'], 
            columns=cc_scoring['debt']).reset_index().rename(columns={0: 'loan_paid', 
                                                                              1:'loan_default'}
                                                            )
res['percent_paid'] = 100 * res['loan_paid'] / (res['loan_paid'] + res['loan_default'])

In [ ]:
res

debt,family_status,loan_paid,loan_default,percent_paid
0,civil partnership,3789,388,90.711037
1,divorced,1110,85,92.887029
2,married,11449,931,92.479806
3,unmarried,2539,274,90.259509
4,widow / widower,897,63,93.437500


**Conclusión:**


Todo lo que puedo concluir de esto es que la categoría de `widow / widower` es relativamente mejor que las otras categorías en el pago de sus deudas a tiempo. Relativamente, las categorías de `civil partnership` y `unmarried` tienen una probabilidad un poco menor de pagar a tiempo. Necesitamos más datos para determinar realmente qué grupo tiene una mayor probabilidad de incumplimiento.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [ ]:
cc_scoring.groupby('debt')['total_income'].mean()

debt
0    26838.627696
1    26175.066796
Name: total_income, dtype: float64

**Conclusión**

A juzgar por el ingreso medio en ambas categorías, parece sugerir que no hay una distinción clara en "total_income" de deudores vs no deudores. 

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [ ]:
res = pd.crosstab(index=cc_scoring['loan_category'], 
            columns=cc_scoring['debt']).reset_index().rename(columns={0: 'loan_paid', 
                                                                              1:'loan_default'}
                                                            )
res['percent_paid'] = 100 * res['loan_paid'] / (res['loan_paid'] + res['loan_default'])

In [ ]:
res

debt,loan_category,loan_paid,loan_default,percent_paid
0,Education loan,3652,370,90.800597
1,Housing loan,10058,782,92.785978
2,Personal loan,2162,186,92.078365
3,Vehicle loan,3912,403,90.660487


**Conclusión**

Aquí también, podemos ver que los prestatarios en las categorías de `Personal loan` y`Housing loan	` son más rápidos en el pago de sus préstamos en comparación con los de las secciones `Education loan	` y `Vehicle loan`.


# Conclusión general 

1. La educación más común es la educación secundaria con un 70%. 
2. El tipo de ingreso más popular es el empleado que representa el 51% de la población. 
3. El 58% de la población está casada. 
4. Después de la limpieza, la media de días_empleados oscila entre 3 y 44 años. 
3. Después de limpiar, la media de los ingresos totales es de 26700 por mes. 
4. Con base en las observaciones anteriores, parece que ningún campo tiene un impacto significativo en el reembolso del préstamo. Tal vez tengan que ser vistos colectivamente. Además, este conjunto de datos puede carecer de características importantes, como puntaje crediticio, historial de préstamos anteriores, ubicación, tamaño del préstamo, duración, etc., lo que puede tener un mayor impacto en el pago del préstamo.





[Tabla de contenidos](#scrollTo=dRqJAPIOCTbn)